### 1. 라이브러리 Import

In [ ]:
import os
from glob import glob
import shutil
import cv2 as cv
import numpy as np
from tqdm import tqdm

# For parse json
import json
import pprint

# For drawing
import random
import matplotlib.pyplot as plt
import pylab
%matplotlib inline

In [ ]:
# 데이터셋 위치
!ls ../../avikus-data

### 2. 폴더 위치 선언

In [ ]:
# 세그멘테이션 라벨링 파일이 포함된 폴더
labeling_folder = '../../avikus-data/labeling'
rawimg_folder = '../../avikus-data/rawdata/'
include_classes = ["buoy", "boat"] # 파싱하고자 하는 데이터셋

In [ ]:
# 랜덤 컬러
def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

### 3. 샘플 JSON 로드 및 그리기

In [ ]:
# SuperbAI JSON 정보
# Double Polygon, Triple List
# https://ko.docs.superb-ai.com/reference/rotated-boxes-1

In [ ]:
cnt = 0
show_limit = 3

for root, dirs, files in os.walk(labeling_folder):
    if 'json' in dirs:
        fd = os.path.join(root, 'json')
        
        for json_file in glob(os.path.join(fd, "*.json")):
            with open(json_file, "r") as file:
                try:
                    data = json.load(file)
                    
                    # 확인용 디버그 print
                    # print(f"Loaded JSON data from {json_file}:")
                    # pprint.pprint(data)
                    
                    keys = ["objects", "class_name", "annotation", "x", "y"]
                    
                    json_file_basename_without_extension = os.path.splitext(os.path.basename(json_file))[0]
                    image_file_name = json_file_basename_without_extension + ".jpg"
                    image_subfolder_name = json_file.split(os.path.sep)[-3]
                    
                    image_file_path = os.path.join(rawimg_folder, image_subfolder_name, image_file_name)
                    
                    if not os.path.exists(image_file_path):
                        print(f"The file at {image_file_path} does not exist.")
                        break
                    
                    image_array = cv.imread(image_file_path)
                    image_array = cv.cvtColor(image_array, cv.COLOR_BGR2RGB)
                    
                    draw_img = image_array.copy()
                    
                    if keys[0] in data:
                        for object in data[keys[0]]:
                            cls_name = object[keys[1]]
                            
                            if cls_name in include_classes:
                                annos = object[keys[2]] # len=1;
                                color = random_color()
                                
                                # 객체 단위
                                for obj_annos in annos:
                                    polygon = []
                                    # 객체 안에 최외곽선을 포함한 Hole이 존재할 수 있음                            
                                    for obj_anno in obj_annos:
                                        for pts in obj_anno:
                                            x, y = pts[keys[3]], pts[keys[4]]
                                            polygon.append([x, y])
                                            
                                    polygon_xy = np.array(polygon, np.int32)
                                    draw_img = cv.polylines(draw_img, [polygon_xy], True, color, thickness=3)
                                    # draw_img = cv.fillPoly(draw_img, [polygon_xy], (0, 255, 0))
                    
                        plt.figure(figsize=(8, 10))
                        plt.imshow(draw_img)
                        plt.axis('off')
                                
                except json.JSONDecodeError as e:
                    print(f"Error loading JSON file {json_file}: {e}")
            
            cnt = cnt + 1
            
            if cnt >= show_limit:
                break
        
        break

### 4. draw_image에 대해서 모두 저장하기

In [ ]:
# polygon 이미지를 저장하는 폴더
draw_image_save_folder = '../../avikus-data/draw_image/'

# 폴더 생성
if not os.path.exists(draw_image_save_folder):
    os.makedirs(draw_image_save_folder)
    print(f"The folder {draw_image_save_folder} was created.")
else:
    print(f"The folder {draw_image_save_folder} already exists.")

In [ ]:
for root, dirs, files in tqdm(os.walk(labeling_folder)):
    if 'json' in dirs:
        fd = os.path.join(root, 'json')
        
        for json_file in glob(os.path.join(fd, "*.json")):
            with open(json_file, "r") as file:
                try:
                    data = json.load(file)
                    
                    # 확인용 디버그 print
                    # print(f"Loaded JSON data from {json_file}:")
                    # pprint.pprint(data)
                    
                    # 사용할 키
                    keys = ["objects", "class_name", "annotation", "x", "y"]
                    
                    json_file_basename_without_extension = os.path.splitext(os.path.basename(json_file))[0]
                    image_file_name = json_file_basename_without_extension + ".jpg"
                    image_subfolder_name = json_file.split(os.path.sep)[-3]
                    
                    # 실제 이미지 파일 경로 파싱
                    image_file_path = os.path.join(rawimg_folder, image_subfolder_name, image_file_name)

                    # 이미지 형시
                    if not os.path.exists(image_file_path):
                        print(f"The file at {image_file_path} does not exist.")
                        
                        image_file_name = json_file_basename_without_extension + ".png"
                        image_file_path = os.path.join(rawimg_folder, image_subfolder_name, image_file_name)
                        
                        if not os.path.exists(image_file_path):
                            print(f"The file at {image_file_path} does not exist.")
                            
                            break
                    
                    image_array = cv.imread(image_file_path)
                    image_array = cv.cvtColor(image_array, cv.COLOR_BGR2RGB)
                    
                    draw_img = image_array.copy()
                    
                    # Replace 'labeling' with 'draw_image' in the path
                    save_image_path = os.path.splitext(json_file.replace('labeling', 'draw_image'))[0] + '.jpg'

                    # Extract the directory path from the modified path
                    save_image_folder = os.path.dirname(save_image_path)

                    # Create the directory if it doesn't exist
                    if not os.path.exists(save_image_folder):
                        os.makedirs(save_image_folder)
                        print(f"The folder {save_image_folder} was created.")
                        
                    if keys[0] in data:
                        for object in data[keys[0]]:
                            cls_name = object[keys[1]]
                            
                            if cls_name in include_classes:
                                annos = object[keys[2]] # len=1;
                                color = random_color()
                                
                                # 객체 단위
                                for obj_annos in annos:    
                                    polygon = []
                                    # 객체 안에 최외곽선을 포함한 Hole이 존재할 수 있음                            
                                    for obj_anno in obj_annos:
                                        for pts in obj_anno:
                                            x, y = pts[keys[3]], pts[keys[4]]
                                            polygon.append([x, y])
                                            
                                    polygon_xy = np.array(polygon, np.int32)
                                    draw_img = cv.polylines(draw_img, [polygon_xy], True, color, thickness=3)
                                    # draw_img = cv.fillPoly(draw_img, [polygon_xy], (0, 255, 0))
                    
                        cv.imwrite(save_image_path, cv.cvtColor(draw_img, cv.COLOR_RGB2BGR))

                except json.JSONDecodeError as e:
                    print(f"Error loading JSON file {json_file}: {e}")
                    